# Analyse du Yelp Open Dataset
## Exemple d'utilisation des outils d'analyse

Ce notebook démontre les fonctionnalités principales du package `yelp_analysis` :
- Chargement des données Yelp
- Recherche d'établissements
- Analyse des avis
- Visualisations

## 1. Import des bibliothèques

In [ ]:
import sys
from pathlib import Path

# Ajouter le répertoire src au path Python
sys.path.insert(0, str(Path.cwd().parent / 'src'))

from yelp_analysis.data_loader import YelpDataLoader
from yelp_analysis.business_search import BusinessSearch
from yelp_analysis.review_analyzer import ReviewAnalyzer
from yelp_analysis import visualizations as viz

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Configuration de l'affichage
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## 2. Chargement des données

**Note:** Les fichiers du Yelp Open Dataset doivent être téléchargés depuis https://www.yelp.com/dataset
et placés dans le répertoire `data/raw/`

In [ ]:
# Définir le répertoire des données
data_dir = Path.cwd().parent / 'data' / 'raw'

# Initialiser le loader
loader = YelpDataLoader(data_dir)

print(f"Répertoire de données: {data_dir}")
print(f"Fichiers disponibles:")
print(f"  - Business: {loader.business_file.exists()}")
print(f"  - Reviews: {loader.review_file.exists()}")
print(f"  - Users: {loader.user_file.exists()}")

### 2.1 Charger les établissements (businesses)

Pour cet exemple, on limite le chargement à 10,000 établissements.
Supprimez le paramètre `limit` pour charger toutes les données.

In [ ]:
# Charger les données des établissements
businesses = loader.load_businesses(limit=10000)

print(f"Nombre d'établissements chargés: {len(businesses)}")
print(f"\nPremières lignes:")
businesses.head()

In [ ]:
# Informations sur les colonnes
businesses.info()

### 2.2 Charger les avis (reviews)

In [ ]:
# Charger les avis (limité à 50,000 pour l'exemple)
reviews = loader.load_reviews(limit=50000)

print(f"Nombre d'avis chargés: {len(reviews)}")
print(f"\nPremières lignes:")
reviews.head()

## 3. Recherche d'établissements

Démonstration des différentes fonctions de recherche et de filtrage.

In [ ]:
# Initialiser le moteur de recherche
search = BusinessSearch(businesses)

### 3.1 Recherche par catégorie

In [ ]:
# Rechercher des restaurants
restaurants = search.search_by_category("Restaurants")
print(f"Nombre de restaurants trouvés: {len(restaurants)}")

# Afficher les 5 meilleurs
restaurants.nlargest(5, 'stars')[['name', 'city', 'stars', 'review_count']]

In [ ]:
# Rechercher des hôtels
hotels = search.search_by_category("Hotels")
print(f"Nombre d'hôtels trouvés: {len(hotels)}")

hotels.nlargest(5, 'stars')[['name', 'city', 'stars', 'review_count']]

### 3.2 Filtrage par note

In [ ]:
# Établissements avec 5 étoiles
five_stars = search.filter_by_rating(min_stars=5.0, max_stars=5.0)
print(f"Nombre d'établissements 5 étoiles: {len(five_stars)}")

# Établissements avec note entre 4 et 5
highly_rated = search.filter_by_rating(min_stars=4.0, max_stars=5.0)
print(f"Nombre d'établissements bien notés (4-5 étoiles): {len(highly_rated)}")

### 3.3 Top établissements par catégorie

In [ ]:
# Top 10 restaurants
top_restaurants = search.get_top_rated(n=10, category="Restaurants")
top_restaurants[['name', 'city', 'state', 'stars', 'review_count']]

## 4. Analyse des avis

Analyse statistique des avis et des notes.

In [ ]:
# Initialiser l'analyseur d'avis
analyzer = ReviewAnalyzer(reviews)

### 4.1 Distribution des notes

In [ ]:
# Distribution des étoiles
distribution = analyzer.get_rating_distribution()
print("Distribution des notes (1-5 étoiles):")
print(distribution)
print(f"\nNote moyenne: {analyzer.get_average_rating():.2f} étoiles")

### 4.2 Statistiques sur les textes

In [ ]:
# Statistiques sur la longueur des avis
text_stats = analyzer.get_text_length_stats()
print("Statistiques sur la longueur des avis (en caractères):")
for key, value in text_stats.items():
    print(f"  {key}: {value:.0f}")

### 4.3 Analyse par note

In [ ]:
# Analyse de sentiment par note
sentiment_analysis = analyzer.analyze_sentiment_by_rating()
sentiment_analysis

### 4.4 Avis les plus utiles

In [ ]:
# Top 5 avis les plus utiles
most_useful = analyzer.get_most_useful_reviews(n=5)
most_useful[['stars', 'useful', 'text']].head()

## 5. Visualisations

### 5.1 Distribution des étoiles

In [ ]:
viz.plot_rating_distribution(reviews['stars'])
plt.show()

### 5.2 Top catégories d'établissements

In [ ]:
viz.plot_business_categories(businesses, top_n=15)
plt.show()

### 5.3 Relation entre nombre d'avis et note moyenne

In [ ]:
viz.plot_rating_vs_review_count(businesses)
plt.show()

### 5.4 Évolution des avis au fil du temps

In [ ]:
viz.plot_reviews_over_time(reviews)
plt.show()

## 6. Conclusion

Ce notebook a démontré les fonctionnalités principales du package `yelp_analysis` :

1. **Chargement de données** : Import efficace des données Yelp au format JSON
2. **Recherche d'établissements** : Recherche par nom, catégorie, note et localisation
3. **Analyse des avis** : Distribution des notes, statistiques, avis les plus utiles
4. **Visualisations** : Graphiques pour explorer les données

Ces outils constituent une base solide pour des analyses plus avancées avec machine learning et deep learning.